In [ ]:
import pandas as pd
import json

In [ ]:
# load sets to list from json file
file_name = 'scraped_BE_final.json'

with open(file_name, 'r') as file:
    json_data = json.load(file)

index_ls = []
set_ls = []
for json_set in json_data:
    name = json_set.keys()
    set_info = json_set.values()

    index_ls.append(*name)
    set_ls.append(*set_info)

In [ ]:
# transform nested dictionaries to flatten dictionary 
def flatten_dict_set_info(dictionary: dict) -> dict:
    temp_dict = {}

    theme = dictionary['set_info'].get('theme', None)
    year = dictionary['set_info'].get('year', None)
    av = dictionary['set_info'].get('availability', None)
    pieces = dictionary['set_info'].get('Pieces', None)
    minifigs = dictionary['set_info'].get('Minifigs', None)

    temp_dict['Theme'] = theme
    temp_dict['Year'] = year
    temp_dict['Availability'] = av
    temp_dict['Pieces'] = pieces
    temp_dict['Minifigs'] = minifigs

    return temp_dict

def flatten_dict_prices(dictionary: dict) -> dict:
    temp_dict = {}

    retail = dictionary['prices'].get('Retail', None)
    value = dictionary['prices'].get('Value', None)

    temp_dict['Retail'] = retail
    temp_dict['Value'] = value

    return temp_dict

def convert_to_dict(stores: list) -> dict:
    temp_dict = {'LEGO': 0, 'Amazon': 0, 'Bricklink': 0, 'StockX': 0, 'eBay': 0}

    for store in stores:
        match store:
            case 'LEGO':
                temp_dict['LEGO'] = 1
            case 'StockX':
                temp_dict['StockX'] = 1
            case 'Amazon':
                temp_dict['Amazon'] = 1
            case 'Bricklink':
                temp_dict['Bricklink'] = 1
            case _:
                temp_dict['eBay'] = 1 
    return temp_dict

list_to_df = []
error_ls = [] # for potential missing informations -> finally only set_info
for num, lego_set in enumerate(set_ls):
    set_dict = {}

    try:
        temp = flatten_dict_set_info(lego_set)
        set_dict.update(temp)
    except:
        print(f'An error occurred for set_info - iteration number: {num}')
        error_ls.append(num)

    try:
        temp = flatten_dict_prices(lego_set)
        set_dict.update(temp)
    except:
        print(f'An error occurred for prices - iteration number: {num}')
        error_ls.append(num)
    
    try:
        temp = convert_to_dict(lego_set['stores'])
        set_dict.update(temp)
    except:
        print(f'An error occurred for stires - iteration number: {num}')
        error_ls.append(num)

    list_to_df.append(set_dict)

In [ ]:
# adding manually missing values
print(error_ls)
print(list_to_df[error_ls[0]])

errored_set = list_to_df[error_ls[0]]
temp = {'Theme': 'Value Packs', 'Year': 2018, 'Availability': 'Retired', 'Pieces': 0, 'Minifigs': 0}
errored_set.update(temp)

list_to_df[error_ls[0]] = errored_set

# I'll remove Value Packs anyway...

In [ ]:
# convert list of dicts to dataframe

set_df = pd.DataFrame(list_to_df, index=index_ls)
set_df # check missing values using Data Wrangler

# missing pieces -> promotional minifig -> can be set to zero
# missinc minifigs -> should be set to zero

# Change Availability column to numeric
qry = (set_df['Availability'] == 'Exclusive')
set_df['Exclusive'] = 0
set_df.loc[qry, 'Exclusive'] = 1

# Fill missing pieces
set_df.loc[set_df['Pieces'].isna(), 'Pieces'] = 0

# Fill missing minifigs
set_df.loc[set_df['Minifigs'].isna(), 'Minifigs'] = 0

# Fill promotional sets with retail zero and create promotional column
qry = (set_df['Retail'].isin(['Promotional', 'Promotional or Unknown']))
set_df['Promotional'] = 0
set_df.loc[qry, 'Promotional'] = 1
set_df.loc[qry, 'Retail'] = 0.00

# Fill value of sets on market
qry = (set_df['Value'].isin(['Not yet released', 'Available at retail']))
set_df.loc[qry, 'Value'] = set_df.loc[qry, 'Retail']

# Transform Year Column
set_df['Years'] = 2025 - set_df['Year']

# remove  Value Packs - mostly missing data about minifigs and pieces - it should be included - sets are based on 2 or more different sets...
qry = (set_df['Theme'].str.strip() != 'Value Packs')
set_df = set_df[qry]

# Drop columns and change datatype
set_df.drop(columns=['Theme', 'Year', 'Availability'], inplace=True)
set_df['Minifigs'] = set_df['Minifigs'].astype(int)
set_df['Retail'] = set_df['Retail'].astype(float)
set_df['Value'] = set_df['Value'].astype(float)

In [ ]:
# Check head
set_df.head(n= 15)

In [ ]:
# Check dtypes
set_df.dtypes

In [ ]:
# write to parquet
file_out_name = 'clean_BE_df.parquet'

set_df.to_parquet(file_out_name)